## Simple Implementation: DeepONet

### Download Data

In [ ]:
# !gdown --id 1SC1TEgwLhhqZ4eAzpcs_X-8Cde1-An-c    # Darcy from FNO, Li el al. 2021


### Initilization

1. Loading

In [ ]:
from tqdm import tqdm
import torch
import torch.nn as nn
import scipy
import numpy as np
import random
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import MultiStepLR, CosineAnnealingLR

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
seed = 8848
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # If using multi-GPU
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

2. Helper functions/classes

In [ ]:
class DenseNet(nn.Module):
    """
    A fully connected neural network (MLP) with ReLU activations between layers, except the last one.
    """
    def __init__(self, layers):
        super(DenseNet, self).__init__()

        self.n_layers = len(layers) - 1
        assert self.n_layers >= 1
        self.layers = nn.ModuleList()
        for j in range(self.n_layers):
            self.layers.append(nn.Linear(layers[j], layers[j+1]))
            if j != self.n_layers - 1:
                self.layers.append(nn.ReLU())

    def forward(self, x):
        for _, l in enumerate(self.layers):
            x = l(x)
        return x

In [ ]:
class L2Loss(object):
    """Computes the relative L2 loss between two tensors without size averaging."""
    def __init__(self):
        super(L2Loss, self).__init__()

    def __call__(self, x, y):
        num_examples = x.size()[0]
        diff_norms = torch.norm(x.reshape(num_examples, -1) - y.reshape(num_examples, -1), p=2, dim=1)
        y_norms = torch.norm(y.reshape(num_examples, -1), p=2, dim=1)
        return torch.sum(diff_norms / y_norms)

In [ ]:
def torch2dgrid(num_x, num_y, bot=(0,0), top=(1,1)):
    """
    Generates a 2D grid of points as a PyTorch tensor within the specified bounds.
    """
    x_bot, y_bot = bot
    x_top, y_top = top
    x_arr = torch.linspace(x_bot, x_top, steps=num_x)
    y_arr = torch.linspace(y_bot, y_top, steps=num_y)
    xx, yy = torch.meshgrid(x_arr, y_arr, indexing='ij')
    mesh = torch.stack([xx, yy], dim=2)
    return mesh

3. reading and loading data

In [ ]:
# reading data
data = scipy.io.loadmat('darcy.mat')
a = data["coeff"]
u = data["sol"]
s = 49 # resolution of output data
print(a.shape, u.shape)

(1024, 49, 49) (1024, 49, 49)


In [ ]:
# loading data
n_train = 896
n_test = 128
batch_size = 32
a_train = torch.Tensor(a[:n_train]).to(device)
u_train = torch.Tensor(u[:n_train]).to(device)
a_test = torch.Tensor(a[-n_test:]).to(device)
u_test = torch.Tensor(u[-n_test:]).to(device)
dataloader = DataLoader(torch.utils.data.TensorDataset(a_train, u_train), batch_size=batch_size, shuffle=True)
dataloader_test = DataLoader(torch.utils.data.TensorDataset(a_test, u_test), batch_size=batch_size, shuffle=False)

# creating the output observation locations for the trunk
grid = torch2dgrid(s, s, bot=(0, 0), top=(1, 1))
grid = grid.reshape(-1, 2).to(device)

a_num_points = s * s # number of observation/sensor points for the input
d = 2 # dimension of the output function

In [ ]:
a_train.shape, u_train.shape, a_test.shape, u_test.shape

(torch.Size([896, 49, 49]),
 torch.Size([896, 49, 49]),
 torch.Size([128, 49, 49]),
 torch.Size([128, 49, 49]))

### DeepONet and Training

4. Define the DeepONet architecture

In [ ]:
class DeepONet(nn.Module):
    def __init__(self, branch_layer, trunk_layer):
        super(DeepONet, self).__init__()
        self.branch = DenseNet(branch_layer)
        self.trunk = DenseNet(trunk_layer)

    def forward(self, a, grid):
        b = self.branch(a)
        t = self.trunk(grid)
        return torch.einsum('bp,np->bn', b, t)

branch_layers = [250, 250, 250, 250, 250, 250, 250, 250]
trunk_layers = [250, 250, 250, 250, 250, 250, 250, 250]
model = DeepONet(branch_layer=[a_num_points] + branch_layers,
                   trunk_layer=[d] + trunk_layers).to(device)

5. training

In [ ]:
optimizer = Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=2000)
epochs = range(2000)
pbar = epochs
pbar = tqdm(pbar, dynamic_ncols=True, smoothing=0.1)
myloss = L2Loss()

for e in pbar:
    train_loss = 0.0
    model.train()
    for a, u in dataloader:
        pred = model(a.reshape(batch_size, -1), grid)
        loss = myloss(pred.reshape(batch_size, s, s), u)
        model.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= n_train
    scheduler.step()

    test_loss = 0.0
    model.eval()
    with torch.no_grad():
        for a, u in dataloader_test:
            pred = model(a.reshape(batch_size, -1), grid)
            loss = myloss(pred.reshape(batch_size, s, s), u)
            test_loss += loss.item()
        test_loss /= n_test
        scheduler.step()

    pbar.set_description(
        (
            f'Epoch: {e}; Averaged train loss: {train_loss:.5f}; Averaged test loss: {test_loss:.5f}; '
        )
    )

Epoch: 1999; Averaged train loss: 0.03224; Averaged test loss: 0.04368; : 100%|██████████| 2000/2000 [05:20<00:00,  6.24it/s]


Note: This is a very simple implementation of DeepONet. More advanced designs can lead to much better performance.